#Instalando dependências

In [35]:
!pip install neo4j

#Inserindo o JSON base

In [36]:
from google.colab import files
uploaded = files.upload()

Saving pokedex.json to pokedex (3).json


#Criando a conexão com o Neo4j

In [37]:
import json
from neo4j import GraphDatabase

# Conectando ao Neo4J online
uri = "neo4j+s://4edc86b4.databases.neo4j.io"
username = "neo4j"
password = "cH-m7R7VxnFEDK6WDDIl2TmpdMfOGkWu_NEITEV4_Xc"

In [38]:
# Inicializando a conexão com o Neo4J online
driver = GraphDatabase.driver(uri, auth=(username, password))

# Função para lidar com a leitura do arquivo JSON
def carregar_pokemons(arquivo_json):
    try:
        with open(arquivo_json, encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Arquivo {arquivo_json} não encontrado.")
    except json.JSONDecodeError:
        print("Erro ao decodificar o arquivo JSON.")
    return []

# Função para criar ou atualizar o nó do Pokémon
def criar_no_pokemon(tx, pokemon):
    habilidades = [hab['nome'] for hab in pokemon.get('habilidades', [])]

    tx.run("""
    MERGE (p:Pokemon {numero: $numero})
    SET p.nome = $nome, p.tipos = $tipos, p.peso = $peso, p.habilidades = $habilidades
    """,
    numero=pokemon['numero'],
    nome=pokemon['nome'],
    tipos=pokemon['tipos'],
    peso=pokemon['peso'],
    habilidades=habilidades)

# Função para criar as arestas de evolução entre Pokémons
def criar_arestas_evolucao(tx, pokemon):
    evolucoes = pokemon.get('evolucoes', [])
    for evolucao in evolucoes:
        tx.run("""
        MATCH (p1:Pokemon {numero: $numero}), (p2:Pokemon {numero: $evolucao})
        MERGE (p1)-[:EVOLUI_PARA]->(p2)
        """, numero=pokemon['numero'], evolucao=evolucao['numero'])

# Função para criar os nós de habilidades e conectá-las ao Pokémon
def criar_habilidades(tx, pokemon):
    habilidades = [hab['nome'] for hab in pokemon.get('habilidades', [])]

    for habilidade in habilidades:
        # Criar o nó da habilidade
        tx.run("""
        MERGE (h:Habilidade {nome: $nome})
        """, nome=habilidade)

        # Conectar Pokémon à habilidade
        tx.run("""
        MATCH (p:Pokemon {numero: $numero}), (h:Habilidade {nome: $nome})
        MERGE (p)-[:TEM_HABILIDADE]->(h)
        """, numero=pokemon['numero'], nome=habilidade)

# Função para criar os nós de tipo e conectá-los ao Pokémon
def criar_tipos(tx, pokemon):
    tipos = pokemon.get('tipos', [])

    for tipo in tipos:
        # Criar o nó do tipo, se não existir
        tx.run("""
        MERGE (t:Tipo {nome: $nome})
        """, nome=tipo)

        # Conectar Pokémon ao tipo
        tx.run("""
        MATCH (p:Pokemon {numero: $numero}), (t:Tipo {nome: $nome})
        MERGE (p)-[:TEM_TIPO]->(t)
        """, numero=pokemon['numero'], nome=tipo)

# Função principal para adicionar todos os detalhes do Pokémon ao Neo4J
def adicionar_pokemon(tx, pokemon):
    criar_no_pokemon(tx, pokemon)
    criar_arestas_evolucao(tx, pokemon)
    criar_habilidades(tx, pokemon)
    criar_tipos(tx, pokemon)

# Função para inserir todos os Pokémons no Neo4J
def inserir_pokemons_no_neo4j(driver, arquivo_json):
    pokemons = carregar_pokemons(arquivo_json)

    if not pokemons:
        return

    with driver.session() as session:
        for pokemon in pokemons:
            session.execute_write(adicionar_pokemon, pokemon)

# Carregar e inserir todos os Pokémons no Neo4J
inserir_pokemons_no_neo4j(driver, './pokedex.json')

driver.close()

#Consultas e agregações
- Quais Pokémons podem atacar um Pikachu pelo sua fraqueza (tipo que lhe pode causar mais dano) cujo o peso é mais de 10kg?
- Quantas segundas e terceiras evoluções que fazem um Pokémon no mínimo dobrar de peso?

In [39]:
driver = GraphDatabase.driver(uri, auth=(username, password))

def buscar_pokemons_por_fraqueza_e_peso(tx):
    query = """
    MATCH (p:Pokemon)-[:TEM_TIPO]->(t:Tipo)
    WHERE t.nome IN ['Ground'] AND p.peso > 10
    RETURN p.nome AS Pokemon, p.peso AS Peso
    """
    result = tx.run(query)
    for record in result:
        print(f"Pokémon: {record['Pokemon']}, Peso: {record['Peso']}kg")

with driver.session() as session:
    session.execute_read(buscar_pokemons_por_fraqueza_e_peso)

driver.close()

Pokémon: Ting-Lu, Peso: 699.7kg
Pokémon: Iron Treads, Peso: 240.0kg
Pokémon: Sandy Shocks, Peso: 60.0kg
Pokémon: Clodsire, Peso: 223.0kg
Pokémon: Great Tusk, Peso: 320.0kg
Pokémon: Toedscool, Peso: 33.0kg
Pokémon: Toedscruel, Peso: 58.0kg
Pokémon: Ursaluna, Peso: 290.0kg
Pokémon: Runerigus, Peso: 66.6kg
Pokémon: Sandaconda, Peso: 65.5kg
Pokémon: Sandygast, Peso: 70.0kg
Pokémon: Palossand, Peso: 250.0kg
Pokémon: Mudbray, Peso: 110.0kg
Pokémon: Mudsdale, Peso: 920.0kg
Pokémon: Zygarde, Peso: 305.0kg
Pokémon: Diggersby, Peso: 42.4kg
Pokémon: Landorus, Peso: 68.0kg
Pokémon: Golett, Peso: 92.0kg
Pokémon: Golurk, Peso: 330.0kg
Pokémon: Stunfisk, Peso: 11.0kg
Pokémon: Sandile, Peso: 15.2kg
Pokémon: Krookodile, Peso: 96.3kg
Pokémon: Krokorok, Peso: 33.4kg
Pokémon: Seismitoad, Peso: 62.0kg
Pokémon: Palpitoad, Peso: 17.0kg
Pokémon: Excadrill, Peso: 40.4kg
Pokémon: Mamoswine, Peso: 291.0kg
Pokémon: Gliscor, Peso: 42.5kg
Pokémon: Rhyperior, Peso: 282.8kg
Pokémon: Hippowdon, Peso: 300.0kg
Pokémon: 

In [40]:
driver = GraphDatabase.driver(uri, auth=(username, password))

# Função de consulta: Segundas e terceiras evoluções que dobram o peso
def buscar_evolucoes_que_dobram_peso(tx):
    query = """
    MATCH (p1:Pokemon)-[:EVOLUI_PARA]->(p2:Pokemon)
    WHERE p2.peso >= 2 * p1.peso
    RETURN COUNT(p2) AS Evolucoes_que_Dobraram_Peso
    """
    result = tx.run(query)
    for record in result:
        print(f"Evoluções que dobraram o peso: {record['Evolucoes_que_Dobraram_Peso']}")

# Executar a consulta
with driver.session() as session:
    session.execute_read(buscar_evolucoes_que_dobram_peso)

driver.close()

Evoluções que dobraram o peso: 391


#Fechamento da conexão

In [41]:
driver.close()